<a href="https://colab.research.google.com/github/shaya-686/AI/blob/main/%D0%94%D0%97_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання

* Створіть Reader
* Створіть датасет та розділіть його на тренувальні та тестові дані
* Виберіть метрики для поріняння якості моделей
* На основі метрик виберіть найкращу модель



In [1]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module7/data/ratings.csv")

df.head()

,user_id,movie_id,rating,timestamp
0,172,94969,5.0,1396067836
1,172,98956,4.0,1396067879
2,176,73881,4.0,1499807147
3,221,1900,4.5,1288550866
4,333,33688,4.0,1412015122


In [3]:
df.shape

(17604, 4)

In [5]:
df.describe()

,user_id,movie_id,rating,timestamp
count,17604.000000,17604.000000,17604.000000,1.760400e+04
mean,141251.609009,88913.605942,3.465945,1.403256e+09
std,81731.882435,51285.238375,1.203144,1.515083e+08
min,172.000000,139.000000,0.500000,8.282224e+08
25%,71256.000000,55098.000000,3.000000,1.395695e+09
50%,138487.000000,89554.000000,3.500000,1.456552e+09
75%,210264.500000,134779.500000,4.500000,1.494418e+09
max,283195.000000,193843.000000,5.000000,1.537934e+09


In [4]:
!pip install -q surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [7]:
from surprise import Dataset, SVD, Reader

reader = Reader(rating_scale=(0.5, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[["user_id", "movie_id","rating"]], reader)

In [8]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, train_size=0.8)

In [9]:
from surprise import BaselineOnly

bsl_options = {'method': 'sgd',
               'reg_u': 0.0001,
               'reg_i': 0.0001}

algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(trainset)

Estimating biases using sgd...


In [10]:
from surprise import accuracy

In [11]:
preds = algo.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")
print(f"fcp = {accuracy.fcp(preds, verbose=False)}")

mae = 0.7386838888671069
mse = 0.9333099840675722
rmse= 0.9660796986106126
fcp = 0.61628864524656


In [12]:
from surprise import SVD

algo = SVD(n_factors=150,
           n_epochs=50,
           )

algo.fit(trainset)

In [13]:
preds = algo.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")
print(f"fcp = {accuracy.fcp(preds, verbose=False)}")

mae = 0.7260633486348896
mse = 0.9187349468409372
rmse= 0.9585066232639904
fcp = 0.5874897736569403


In [14]:
from surprise import KNNBasic

algo = KNNBasic(k=15,
                min_k=5,
                sim_options={'name': 'cosine',
                             'user_based': True})

algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [15]:

preds = algo.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")
print(f"fcp = {accuracy.fcp(preds, verbose=False)}")

mae = 0.9100698449016235
mse = 1.3958767545417994
rmse= 1.18147228259566
fcp = 0.6088315033750209


In [16]:
from surprise import CoClustering

algo = CoClustering(n_cltr_u = 5,
                    n_cltr_i = 15,
                    n_epochs = 40)

algo.fit(trainset)

In [17]:
preds = algo.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")
print(f"fcp = {accuracy.fcp(preds, verbose=False)}")

mae = 0.8090540896551626
mse = 1.1334747578925621
rmse= 1.0646477153934828
fcp = 0.6250106798052003


In [18]:
from sklearn import metrics


def objective(trial):


  params = {
        'n_factors': trial.suggest_int('n_factors', 10, 100),
        'n_epochs': trial.suggest_int('n_epochs', 10, 50),
        'lr_all': trial.suggest_float('lr_all', 1e-4, 0.1),
        'reg_all': trial.suggest_float('reg_all', 1e-3, 1.0)
    }


  model = SVD(**params)

  model.fit(trainset)

  preds = model.test(testset)
  mae = accuracy.mae(preds, verbose=False)

  return mae

In [19]:
!pip install -q optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.2 MB/s eta 0:00:00


In [20]:
import optuna
study = optuna.create_study(directions=['minimize'])
study.optimize(objective, n_trials=20)

[I 2024-09-03 19:43:36,605] A new study created in memory with name: no-name-84159196-d403-4a9b-80ba-d743ccdaacde
[I 2024-09-03 19:43:37,306] Trial 0 finished with value: 0.7373462025130577 and parameters: {'n_factors': 60, 'n_epochs': 16, 'lr_all': 0.05081467797979605, 'reg_all': 0.39833438047094694}. Best is trial 0 with value: 0.7373462025130577.
[I 2024-09-03 19:43:38,273] Trial 1 finished with value: 0.7823971170831818 and parameters: {'n_factors': 90, 'n_epochs': 30, 'lr_all': 0.00958426926578862, 'reg_all': 0.8821874979722716}. Best is trial 0 with value: 0.7373462025130577.
[I 2024-09-03 19:43:39,319] Trial 2 finished with value: 0.7268298020240983 and parameters: {'n_factors': 66, 'n_epochs': 46, 'lr_all': 0.06368597900927064, 'reg_all': 0.2826798930157876}. Best is trial 2 with value: 0.7268298020240983.
[I 2024-09-03 19:43:39,853] Trial 3 finished with value: 0.7683844594573991 and parameters: {'n_factors': 10, 'n_epochs': 34, 'lr_all': 0.04894163469083619, 'reg_all': 0.7210

In [21]:
best_params = study.best_trials[0].params
best_params

{'n_factors': 10,
 'n_epochs': 47,
 'lr_all': 0.02183426678644364,
 'reg_all': 0.1634557403185574}

In [22]:
model = SVD(
    n_factors = 10,
    n_epochs = 47,
    lr_all = 0.02183426678644364,
    reg_all = 0.1634557403185574
)


In [23]:
model.fit(trainset)


In [24]:
preds = model.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")
print(f"fcp = {accuracy.fcp(preds, verbose=False)}")

mae = 0.7021195470657511
mse = 0.842725125553658
rmse= 0.91800061304645
fcp = 0.6517600131478246
